Set these two paths

In [1]:
results_path = '/home/mjw/OneDrive/Microgrid MG2Lab/Dataset/JPL_v4/Forecast/Seasonal offline test 2/'
mg2lab_dataset_path = '/home/mjw/OneDrive/Microgrid MG2Lab/Dataset/'

Pandas and some error calcs

In [2]:
import pandas as pd

pd.options.plotting.backend='plotly'

def calc_mae(y:pd.Series,yhat:pd.Series,norm=1):
    return (y-yhat).abs().mean()/norm

def calc_rmse(y:pd.Series,yhat:pd.Series,norm=1):
    return (((y-yhat)**2).mean())**0.5/norm

def calc_skill(true:pd.Series,forecast:pd.Series,benchmark:pd.Series,method='mae',norm=1):
    if method=='mae':
        return 1 - calc_mae(true,forecast,norm)/calc_mae(true,benchmark,norm)
    if method=='rmse':
        return 1 - calc_rmse(true,forecast,norm)/calc_rmse(true,benchmark,norm)

Arrange seasonal data together

In [3]:
seasons = ['winter','spring','summer','autumn']

#df = pd.read_csv('../../test_offline_seasonal_JPL_v4.csv',index_col=0,parse_dates=True,comment='#')
df = pd.read_csv(mg2lab_dataset_path+'JPL_v4/'+'all_JPL_v4.csv',index_col=0,parse_dates=True,comment='#')

df['persist 168h'] = df.power.shift(96*7).values
df['persist 1h'] = df.power.shift(4).values

#df.columns = ['LoadKW']

load = {}
load['winter'] = df['2018-12-3':'2018-12-16']
load['spring'] = df['2019-3-18':'2019-3-31']
load['summer'] = df['2019-8-5':'2019-8-18']
load['autumn'] = df['2019-10-14':'2019-10-27']
    
forecast = {}
for season in seasons:
    forecast[season] = pd.read_csv(f'{results_path}test_offline_{season}_v2.csv',index_col=1,parse_dates=True,comment='#')
    forecast[season].timestamp_utc = pd.to_datetime(forecast[season].timestamp_utc)
    
for season in seasons:
    load[season].index = pd.date_range(forecast[season].index[0],periods=len(load[season]),freq='15min')


Calculate and plot daily errors

In [4]:
start_dates_load = {'winter':'2018-12-3','summer':'2019-8-5','spring':'2019-3-18','autumn':'2019-10-14'}
start_dates_solar = {'winter':'2023-1-9','summer':'2022-7-4','spring':'2023-4-3','autumn':'2022-10-3'}

Re-run notebook from beginning (or winter week 1 doesn't calculate right)

In [5]:
for season in ['winter','summer','spring','autumn']:
    for week in [0,1]:
        #week=0 # 0 or 1
        update_h = 24 # 1 or 24
        persist_lag_h = 168 # 1 or 168

        t_range = pd.date_range(start_dates_solar[season],periods=7*int(24/update_h),freq=f'{update_h}h') + pd.Timedelta(days=week*7)
        df = pd.DataFrame()
        for t in t_range:
            times_next_day = pd.date_range(t,periods=4*update_h,freq='15min')
            forecast_next_day = forecast[season].loc[t:t+pd.Timedelta('1d')][:4*update_h].predicted_activepower_ev_1.values
            next_day = pd.DataFrame({'Forecast kW':forecast_next_day},
                                    index=times_next_day)
            df = pd.concat([df,next_day])

        df['Load kW'] = load[season]['power']
        df[f'Persist {persist_lag_h}h kW'] = load[season][f'persist {persist_lag_h}h']
        df.index = pd.date_range(start_dates_load[season],periods=7*96,freq='15min') + pd.Timedelta(days=week*7)

        err = {'LSTM nMAE %':[],'LSTM nRMSE %':[],'Persist nMAE %':[],'Persist nRMSE %':[],'LSTM Skill (nMAE) %':[],'LSTM Skill (nRMSE) %':[]}
        err_index = []
        for day in df.index.day.unique():
            df_day = df.loc[df.index.day==day]
            err_index.append(f'{df_day.index[0].month_name()[:3]} {df_day.index[0].day}')
            err['LSTM nMAE %'].append(100*calc_mae(df_day['Load kW'],df_day['Forecast kW'],norm=25))
            err['LSTM nRMSE %'].append(100*calc_rmse(df_day['Load kW'],df_day['Forecast kW'],norm=25))
            err['Persist nMAE %'].append(100*calc_mae(df_day['Load kW'],df_day[f'Persist {persist_lag_h}h kW'],norm=25))
            err['Persist nRMSE %'].append(100*calc_rmse(df_day['Load kW'],df_day[f'Persist {persist_lag_h}h kW'],norm=25))
            err['LSTM Skill (nMAE) %'].append(100*calc_skill(df_day['Load kW'],df_day['Forecast kW'],df_day[f'Persist {persist_lag_h}h kW'],method='mae',norm=25))
            err['LSTM Skill (nRMSE) %'].append(100*calc_skill(df_day['Load kW'],df_day['Forecast kW'],df_day[f'Persist {persist_lag_h}h kW'],method='rmse',norm=25))

        err = pd.DataFrame(err,index=err_index)

        for col in err.columns:
            err[col] = err[col].round(1)
            
        f = df.plot(title=f'<b>LSTM Forecast: {season} week {week+1}</b><br>LSTM {update_h} h update, persist {persist_lag_h} h lag',width=1000)
        f.show()

        print(err.T)
    

                      Dec 3  Dec 4  Dec 5  Dec 6  Dec 7  Dec 8  Dec 9
LSTM nMAE %             5.7    4.2    4.5    4.7    5.0    1.4    2.4
LSTM nRMSE %            8.8    6.5    6.8    6.9    7.8    2.0    3.3
Persist nMAE %          9.2    3.5    3.6    3.9   14.2    1.0    0.9
Persist nRMSE %        14.5    5.7    6.0    6.2   23.9    1.7    1.7
LSTM Skill (nMAE) %    38.6  -19.5  -24.5  -20.0   64.8  -48.1 -172.0
LSTM Skill (nRMSE) %   39.3  -13.5  -13.0  -10.9   67.3  -19.7  -91.0


                      Dec 10  Dec 11  Dec 12  Dec 13  Dec 14  Dec 15  Dec 16
LSTM nMAE %              6.5     5.8     4.9     3.7     9.9     1.2     2.0
LSTM nRMSE %             9.1     7.9     6.9     5.2    16.7     1.8     3.0
Persist nMAE %           4.3     4.1     4.1     3.6    13.9     2.0     0.9
Persist nRMSE %          7.2     6.8     6.6     5.6    23.9     3.2     2.0
LSTM Skill (nMAE) %    -50.9   -41.0   -20.4    -1.9    28.8    38.4  -113.0
LSTM Skill (nRMSE) %   -27.3   -16.0    -4.9     7.0    30.2    43.9   -46.8


                      Aug 5  Aug 6  Aug 7  Aug 8  Aug 9  Aug 10  Aug 11
LSTM nMAE %             6.1    3.7    4.4    9.2   11.4     1.5     2.3
LSTM nRMSE %            9.8    5.2    5.9   13.1   16.9     2.1     3.4
Persist nMAE %          4.0    3.6    4.9    5.9   19.9     1.0     1.9
Persist nRMSE %         5.8    5.1    7.1    9.0   28.9     1.9     3.0
LSTM Skill (nMAE) %   -51.7   -3.2    9.2  -54.7   43.0   -57.4   -19.0
LSTM Skill (nRMSE) %  -68.5   -1.6   16.5  -46.0   41.5   -10.0   -14.6


                      Aug 12  Aug 13  Aug 14  Aug 15  Aug 16  Aug 17  Aug 18
LSTM nMAE %              4.8     3.2     4.9     6.8     9.1     1.5     2.0
LSTM nRMSE %             8.3     5.2     7.3    10.0    13.0     2.1     2.9
Persist nMAE %           3.4     4.3     5.4     8.5    19.7     1.2     0.8
Persist nRMSE %          5.4     6.2     7.0    14.6    27.9     2.1     1.7
LSTM Skill (nMAE) %    -42.2    24.6     8.4    20.7    53.8   -19.8  -167.2
LSTM Skill (nRMSE) %   -54.6    16.0    -5.0    31.1    53.5    -0.0   -71.2


                      Mar 18  Mar 19  Mar 20  Mar 21  Mar 22  Mar 23  Mar 24
LSTM nMAE %              6.3     5.5     5.0     5.5    12.1     2.0     1.5
LSTM nRMSE %             9.5     8.6     7.1     8.2    18.8     3.0     2.3
Persist nMAE %           5.6     3.9     3.1     6.7    20.0     2.0     1.7
Persist nRMSE %          8.8     6.6     4.7    10.6    30.5     3.0     2.8
LSTM Skill (nMAE) %    -11.9   -41.5   -61.7    18.5    39.5     0.5    12.4
LSTM Skill (nRMSE) %    -7.8   -30.0   -51.0    22.6    38.2    -0.9    17.3


                      Mar 25  Mar 26  Mar 27  Mar 28  Mar 29  Mar 30  Mar 31
LSTM nMAE %              7.0     5.7     5.9     4.7     4.7     1.5     2.0
LSTM nRMSE %            10.1     8.2     8.1     7.6     8.5     2.6     3.2
Persist nMAE %           6.1     7.8     6.1     5.5    16.5     1.6     1.2
Persist nRMSE %         10.4    12.4    10.8     8.7    26.1     2.1     2.2
LSTM Skill (nMAE) %    -14.7    26.3     3.7    14.4    71.4     4.5   -59.5
LSTM Skill (nRMSE) %     2.5    33.6    25.0    12.4    67.5   -22.2   -42.7


                      Oct 14  Oct 15  Oct 16  Oct 17  Oct 18  Oct 19  Oct 20
LSTM nMAE %              8.3     6.6     5.4     6.9    10.9     1.5     1.4
LSTM nRMSE %            12.6     8.7     7.9    10.1    16.6     2.0     2.0
Persist nMAE %           6.4     4.5     7.6     6.4    20.8     3.0     1.6
Persist nRMSE %         11.2     7.0    11.6    10.2    31.7     4.9     2.2
LSTM Skill (nMAE) %    -30.3   -46.5    28.9    -6.9    47.7    49.6    11.5
LSTM Skill (nRMSE) %   -12.5   -24.8    31.6     0.5    47.6    57.9     9.8


                      Oct 21  Oct 22  Oct 23  Oct 24  Oct 25  Oct 26  Oct 27
LSTM nMAE %              8.1     5.4     6.1     6.2     6.9     2.1     3.6
LSTM nRMSE %            12.9     7.2     7.8     9.0    11.3     3.0     5.1
Persist nMAE %           7.2     9.4     4.8     4.9    17.6     1.6     3.4
Persist nRMSE %         11.8    12.8     6.9     6.9    27.7     2.5     4.9
LSTM Skill (nMAE) %    -12.5    43.2   -25.7   -26.4    60.9   -34.4    -5.1
LSTM Skill (nRMSE) %    -9.6    44.2   -12.9   -30.5    59.2   -19.0    -3.8
